In [1]:
import copy as pyCopy
%pylab inline
import cv2 as cv2
print(cv2.__version__)

Populating the interactive namespace from numpy and matplotlib
4.1.1


In [2]:
def detectBlueJewel(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    channels = []
    channels = cv2.split(hsv, channels);
    # 0 = H, 1 = S, 2 = V

    channels[0] = cv2.inRange(channels[0], 90, 150, channels[0]);
    __,channels[1] = cv2.threshold(channels[1], 140, 255, cv2.THRESH_BINARY);
    __,channels[2] = cv2.threshold(channels[2], 90, 255, cv2.THRESH_BINARY);

    finalMap = cv2.bitwise_and(channels[0], channels[1])
    finalMap = cv2.bitwise_and(finalMap, channels[2])

    floodFillMap = pyCopy.deepcopy(finalMap)

    cv2.floodFill(floodFillMap,None,(0,0),255)

    floodFillMap = cv2.bitwise_not(floodFillMap)

    finalMap = cv2.bitwise_or(finalMap, floodFillMap)

    finalMap = cv2.GaussianBlur( finalMap, (9, 9), 2, 2)
    
    cv2.imshow("blueBinaryMap", finalMap)

    finalMapCircles = None;
    finalMapCircles = cv2.HoughCircles(finalMap, cv2.HOUGH_GRADIENT, 1.2, 100, param1 = 531, param2 = 29)
    
    if finalMapCircles is not None:
        finalMapCircles = np.round(finalMapCircles[0, :]).astype("int")

        for (x,y,r) in finalMapCircles:
            cv2.circle(img, (x, y), r, (255, 255, 0), 4)
            cv2.rectangle(img, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    
    return img

In [3]:
def detectRedJewel(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    channels = []
    channels = cv2.split(hsv, channels);
    # 0 = H, 1 = S, 2 = V
    
    tempH = pyCopy.deepcopy(channels[0])
    
    tempH = cv2.inRange(channels[0], 0, 10, tempH)
    channels[0] = cv2.inRange(channels[0], 160, 180, channels[0]);
    channels[0] = cv2.bitwise_or(channels[0], tempH)
    
    __,channels[1] = cv2.threshold(channels[1], 140, 255, cv2.THRESH_BINARY);
    __,channels[2] = cv2.threshold(channels[2], 90, 255, cv2.THRESH_BINARY);

    finalMap = cv2.bitwise_and(channels[0], channels[1])
    finalMap = cv2.bitwise_and(finalMap, channels[2])

    floodFillMap = pyCopy.deepcopy(finalMap)

    cv2.floodFill(floodFillMap,None,(0,0),255)

    floodFillMap = cv2.bitwise_not(floodFillMap)

    #finalMap = cv2.bitwise_or(finalMap, floodFillMap)

    finalMap = cv2.GaussianBlur( finalMap, (9, 9), 2, 2)
    
    cv2.imshow("redBinaryMap", finalMap)

    finalMapCircles = None;
    finalMapCircles = cv2.HoughCircles(finalMap, cv2.HOUGH_GRADIENT, 1.2, 100, param1 = 300, param2 =37)

    if finalMapCircles is not None:
        finalMapCircles = np.round(finalMapCircles[0, :]).astype("int")
        for (x,y,r) in finalMapCircles:
            cv2.circle(img, (x, y), r, (0, 255, 255), 4)
            cv2.rectangle(img, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    
    return img

In [4]:
def cubeDetector(img, drawnImg = None):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    #cv2.imwrite("flowchart_stuff/hsv.jpg", hsv)
    
    channels = []
    channels = cv2.split(hsv, channels)
    # 0 = H, 1 = S, 2 = V
    
    #cv2.imshow("H", cv2.resize(channels[0], (480, 640)))
    #cv2.imshow("S", cv2.resize(channels[1], (480, 640)))
    #cv2.imshow("V", cv2.resize(channels[2], (480, 640)))
    
    #cv2.imwrite("flowchart_stuff/goldPreS.jpg", channels[1])
    
    channels[0] = cv2.inRange(channels[0], 0, 30, channels[0]);
    channels[1] = cv2.inRange(channels[1], 120, 240, channels[1]);
    channels[2] = cv2.inRange(channels[2], 80, 255, channels[2]);
    
    #cv2.imwrite("flowchart_stuff/goldS.jpg", channels[1])
    
    #cv2.imshow("goldH", cv2.resize(channels[0], (320, 480)))
    #cv2.imshow("goldS", cv2.resize(channels[1], (320, 480)))
    #cv2.imshow("goldV", cv2.resize(channels[2], (320, 480)))

    finalMap = cv2.bitwise_and(channels[0], channels[1])
    finalMap = cv2.bitwise_and(finalMap, channels[2])
    
    cv2.imwrite("flowchart_stuff/goldBinary.jpg", finalMap)

    #floodFillMap = pyCopy.deepcopy(finalMap)

    #cv2.floodFill(floodFillMap,None,(0,0),255)

    #floodFillMap = cv2.bitwise_not(floodFillMap)

    #finalMap = cv2.bitwise_or(finalMap, floodFillMap)

    finalMap = cv2.GaussianBlur( finalMap, (9, 9), 2, 2)
    
    cv2.imwrite("flowchart_stuff/goldBinaryBlur.jpg", finalMap)
    
    #cv2.imshow("cubeBinaryMap", cv2.resize(finalMap, (480, 640)))
    
    finalMapContours = None;
    finalMapContours = cv2.findContours(finalMap, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE, finalMapContours)
    
    if drawnImg is not None:
        retImg = pyCopy.deepcopy(drawnImg)
    else:
        retImg = pyCopy.deepcopy(img)
    
    if finalMapContours is not None:
        polygons = [];
        for contour in finalMapContours[1]:
            if cv2.contourArea(contour) > 5000:
                polygons.append(cv2.approxPolyDP(contour, .05*cv2.arcLength(contour, True), True))
                M = cv2.moments(contour)
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                 
                # draw the contour and center of the shape on the image
                cv2.circle(retImg, (cX, cY), 7, (255, 255, 255), -1)
                
        cv2.drawContours(retImg, polygons, -1, (255, 0, 0), 4)
    
    return retImg

In [5]:
def detectWhiteMineral(img, drawnImg = None):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    channels = []
    channels = cv2.split(hsv, channels);
    # 0 = H, 1 = S, 2 = V
    
    #cv2.imshow("Hpre", channels[0])
    #cv2.imshow("Spre", channels[1])
    #cv2.imshow("Vpre", channels[2])
    
    cv2.imwrite("flowchart_stuff/whitePreV.jpg", channels[2])
    
    channels[0] = cv2.inRange(channels[0], 0, 200, channels[0]);
    channels[1] = cv2.inRange(channels[1], 0, 20, channels[1]);
    channels[2] = cv2.inRange(channels[2], 160, 255, channels[2]);
    
    cv2.imwrite("flowchart_stuff/whiteV.jpg", channels[2])
    
    #cv2.imshow("whiteH", cv2.resize(channels[0], (320, 480)))
    #cv2.imshow("whiteS", cv2.resize(channels[1], (320, 480)))
    #cv2.imshow("whiteV", cv2.resize(channels[2], (320, 480)))
    
    finalMap = cv2.bitwise_and(channels[0], channels[1])
    finalMap = cv2.bitwise_and(finalMap, channels[2])

    floodFillMap = pyCopy.deepcopy(finalMap)

    #cv2.floodFill(floodFillMap,None,(0,0),255)

    #floodFillMap = cv2.bitwise_not(floodFillMap)

    finalMap = cv2.bitwise_or(finalMap, floodFillMap)
    
    cv2.imwrite("flowchart_stuff/whiteBinaryMap.jpg", finalMap)

    finalMap = cv2.GaussianBlur( finalMap, (9, 9), 2, 2)
    
    cv2.imwrite("flowchart_stuff/whiteBinaryBlur.jpg", finalMap)
    
    cv2.imshow("flowchart_stuff/whiteBinaryMap.jpg", cv2.resize(finalMap, (320, 480)))

    finalMapCircles = None;
    finalMapCircles = cv2.HoughCircles(finalMap, cv2.HOUGH_GRADIENT, .005, 100, param1 = 300, param2 =30)
    
    if drawnImg is not None:
        retImg = pyCopy.deepcopy(drawnImg)
    else:
        retImg = pyCopy.deepcopy(img)

    if finalMapCircles is not None:
        finalMapCircles = np.round(finalMapCircles[0, :]).astype("int")
        for (x,y,r) in finalMapCircles:
            cv2.circle(retImg, (x, y), r, (255, 255, 255), 4)
            cv2.rectangle(retImg, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    
    return retImg

In [6]:
#cap = cv2.VideoCapture(0)
#for i in range (1,2):
if True:
    #ret, frame = cap.read()
    
    frame = cv2.imread(("sampleMinerals/flowchart.jpg"))
    cv2.imwrite("flowchart_stuff/flowchart_final.jpg", detectWhiteMineral(pyCopy.deepcopy(frame), drawnImg = (cubeDetector(pyCopy.deepcopy(frame)))))
     
    #whiteMinerals = detectWhiteMineral(pyCopy.deepcopy(frame))
    #cv2.imshow("Img", cubeDetector(frame, drawnImg = pyCopy.deepcopy(whiteMinerals)))

    key = cv2.waitKey(0) & 0xFF
 
    #if the 'q' key is pressed, stop the loop
    #if key == ord("q"):
        #break

#cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.1) /io/opencv/modules/imgproc/src/shapedescr.cpp:274: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'contourArea'


#### img = cv2.imread('pictures/testing.jpg',1)
blueCircles = detectBlueJewel(pyCopy.deepcopy(img))
cv2.imshow("Frame", detectRedJewel(pyCopy.deepcopy(blueCircles)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [100]:
def detectWhiteMineralContours(img, drawnImg = None):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    channels = []
    channels = cv2.split(hsv, channels);
    # 0 = H, 1 = S, 2 = V
    
    cv2.imshow("H", channels[0])
    cv2.imshow("S", channels[1])
    cv2.imshow("V", channels[2])
    
    channels[0] = cv2.inRange(channels[0], 90, 130, channels[0]);
    channels[1] = cv2.inRange(channels[1], 0, 70, channels[1]);
    channels[2] = cv2.inRange(channels[2], 100, 255, channels[2]);

    finalMap = cv2.bitwise_and(channels[0], channels[1])
    finalMap = cv2.bitwise_and(finalMap, channels[2])

    floodFillMap = pyCopy.deepcopy(finalMap)

    #cv2.floodFill(floodFillMap,None,(0,0),255)

    #floodFillMap = cv2.bitwise_not(floodFillMap)

    #finalMap = cv2.bitwise_or(finalMap, floodFillMap)

    finalMap = cv2.GaussianBlur( finalMap, (9, 9), 2, 2)
    
    cv2.imshow("whiteBinaryMap", finalMap)

    finalMapContours = None;
    finalMapContours = cv2.findContours(finalMap, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE, finalMapContours)
    
    if drawnImg is not None:
        retImg = pyCopy.deepcopy(drawnImg)
    else:
        retImg = pyCopy.deepcopy(img)
    
    if finalMapContours is not None:
        polygons = [];
        for contour in finalMapContours[1]:
            if cv2.contourArea(contour) > 50:
                polygons.append(cv2.approxPolyDP(contour, .0005*cv2.arcLength(contour, True), True))
                
        cv2.drawContours(retImg, polygons, -1, (255, 255, 255), 4)
    
    return retImg